In [1]:
import tkinter as tk
from tkinter import messagebox
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score
import numpy as np

In [2]:
def predict_stock_prices(ticker, start_date, end_date, window_size=30, future_days=30, threshold=0.03):
    # Load stock data
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    stock_data.set_index('Date', inplace=True)

    # Prepare data for model training
    X = []
    y = []
    for i in range(len(stock_data) - window_size):
        X.append(stock_data['Close'].iloc[i:i+window_size].values)
        y.append(stock_data['Close'].iloc[i+window_size])
    X = np.array(X)
    y = np.array(y)

    # Train decision tree model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Convert predicted prices to binary labels (1 for increase, 0 for decrease)
    y_train_labels = (y_train > y_pred_train).astype(int)
    y_test_labels = (y_test > y_pred_test).astype(int)

    # Predict stock prices for next 30 days
    last_window = stock_data['Close'].iloc[-window_size:].values
    predicted_prices = []
    for i in range(future_days):
        predicted_price = model.predict([last_window])[0]
        predicted_prices.append(predicted_price)
        last_window = np.append(last_window[1:], predicted_price)
    future_dates = pd.date_range(start=stock_data.index[-1] + pd.DateOffset(days=1), periods=future_days)
    predicted_prices = np.array(predicted_prices)

    # Calculate F1 score for test set
    y_pred_test_labels = (y_test > y_pred_test).astype(int)
    f1_score_test = f1_score(y_test_labels, y_pred_test_labels)
    print(f1_score_test)

    # Recommend whether to buy or not
    last_actual_price = stock_data['Close'].iloc[-1]
    last_predicted_price = predicted_prices[0]
    if last_predicted_price > last_actual_price * (1 + threshold):
        recommendation = 'BUY'
    elif last_predicted_price < last_actual_price * (1 - threshold):
        recommendation = 'SELL'
    else:
        recommendation = 'HOLD'

    # Create DataFrame for predicted prices
    predicted_prices_df = pd.DataFrame({'Date': future_dates, 'Predicted Price': predicted_prices})

    return predicted_prices_df, recommendation


In [3]:
def get_predictions():
    ticker = ticker_entry.get()
    start_date = start_date_entry.get()
    end_date = end_date_entry.get()
    window_size = int(window_size_entry.get())
    future_days = int(future_days_entry.get())
    threshold = float(threshold_entry.get())

    try:
        predicted_prices_df, recommendation = predict_stock_prices(ticker, start_date, end_date, window_size, future_days, threshold)
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, 'Predicted Prices for Next 30 Days:\n')
        result_text.insert(tk.END, predicted_prices_df.to_string(index=False) + '\n')
        result_text.insert(tk.END, 'Recommendation: ' + recommendation)
    except Exception as e:
        messagebox.showerror('Error', str(e))

# Create tkinter window
root = tk.Tk()
root.title('Stock Price Prediction')
root.geometry('800x600')

# Create input fields
ticker_label = tk.Label(root, text='Ticker Symbol:')
ticker_label.pack()
ticker_entry = tk.Entry(root)
ticker_entry.pack()

start_date_label = tk.Label(root, text='Start Date (YYYY-MM-DD):')
start_date_label.pack()
start_date_entry = tk.Entry(root)
start_date_entry.pack()

end_date_label = tk.Label(root, text='End Date (YYYY-MM-DD):')
end_date_label.pack()
end_date_entry = tk.Entry(root)
end_date_entry.pack()

window_size_label = tk.Label(root, text='Window Size:')
window_size_label.pack()
window_size_entry = tk.Entry(root)
window_size_entry.pack()

future_days_label = tk.Label(root, text='Future Days:')
future_days_label.pack()
future_days_entry = tk.Entry(root)
future_days_entry.pack()

threshold_label = tk.Label(root, text='Threshold:')
threshold_label.pack()
threshold_entry = tk.Entry(root)
threshold_entry.pack()

# Create a submit button
submit_button = tk.Button(root, text='Submit', command=get_predictions)
submit_button.pack()

# Create a text widget for displaying results
result_text = tk.Text(root, height=15, width=80)
result_text.pack()

# Start the tkinter event loop
root.mainloop()

[*********************100%***********************]  1 of 1 completed
1.0
